# Lab 4 – Public Key Certificates and PKI
By Luis Daniel Casais Mezquida  
Cryptography 22/23  
Bachelor's degree in Applied Mathematics and Computing, grp. 121  
Universidad Carlos III de Madrid

## General goal
In this lab you’ll get familiarized with creating and processing X.509 public key certificates and with creating and
operating a simple OpenSSL-based PKI. The lab makes use of the Cryptography library for Python and the OpenSSL
command tool.

## Specific goals
- Deploy a simple one-level demo-CA with OpenSSL
- Generate an end user Certificate Signing Request with pyca/cryptography
- Issue an end user certificate using demo-CA and export it in PEM format with OpenSSL
- Load and unserialize an end user PEM certificate using pyca/cryptography and use the associated public key
- Export a certificate in PKCS12 format and use it to sign a PDF file with Adobe Reader
- See some certificate real world examples  

Modules and tools
The documentation of the pyca/cryptography library can be found [here](https://pypi.org/project/cryptography).

You’ll may find useful to have these links as reference:
- [X.509 Tutorial](https://cryptography.io/en/latest/x509/tutorial.html)
- [X.509 CSR (Certificate Signing Request) Object](https://cryptography.io/en/latest/x509/reference.html#x-509-csr-certificate-signing-request-object)
- [Loading Certificates](https://cryptography.io/en/latest/x509/reference/#loading-certificates)
- [X.509 Certificate Object](https://cryptography.io/en/latest/x509/reference.html#x-509-certificate-object)

[OpenSSL](https://www.openssl.org/) documentation (manpages) can be found [here](https://www.openssl.org/docs/man1.1.1/).  

We are going to use the command line tool, whose commands can be
found [here](https://www.openssl.org/docs/man1.1.1/man1/).  
The commands that you’ll have to use in Lab 4 are the following ones:
- [ca](https://www.openssl.org/docs/man1.1.1/man1/ca.html) - sample minimal CA application
- [x509](https://www.openssl.org/docs/man1.1.1/man1/x509.html) - Certificate display and signing utility
- [req](https://www.openssl.org/docs/man1.1.1/man1/req.html) - PKCS#10 certificate request and certificate generating utility
- [verify](https://www.openssl.org/docs/man1.1.1/man1/verify.html) - Utility to verify certificates

## Lab assignment and assessment

You are given a configuration file (<code>openssl_AC1.cnf</code>) that you will use to specify some options for the Certification Authority considered in the OpenSSL demo-CA.
Next, you’ll be guided through several tasks you’ll have to complete. 
Lab 4 assessment will be performed with a test/quiz/exam at the end of the course.

# 0. Modules installation and imports

In [21]:
import base64

from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key

from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.x509 import load_pem_x509_certificate

# 1. Deploying AC1 and issuing AC1's self-signed certificate
You are going to use the <code>OpenSSL</code> tool to deploy a demo CA. <code>OpenSSL</code> is already installed in the VM you are using and you can use <code>OpenSSL</code> command line tool through the terminal offered in JupyterLab .

1) Create a folder and name it as <code>PKI</code>
2) Open a terminal (File > New > Terminal) and go to the PKI folder by writing the command <code>cd PKI</code> (as shown below in line [0]) and press <code>Enter</code>. 
3) Next, create a new folder for AC1 (line `1`), go into this folder (line `2`) and create the infrastructe needed to deploy AC1 (lines `3`-`4`]). 
4) Place the file <code>openssl_AC1.cnf</code> that you should have downloaded from Aula Global inside the AC1 folder. WITH CARE: You may want to change some data inside the conf file (eg, the common name of the CA)  
5) Now, you are ready to create an RSA key pair and a self-signed certificate for AC1 (line `6`). Below you can see an output similar to the one you should get. NOTE DOWN THE PASSWORD FOR THE PRIVATE KEY if you think that you may forget it (if you do not remember it, you'll have to repeat the process AGAIN if you need to use the private key).

In [ ]:
cd PKI
mkdir AC1
cd AC1
mkdir solicitudes crls nuevoscerts privado
echo '01' > serial
openssl req -x509 -newkey rsa:2048 -days 360 -out ac1cert.pem -outform PEM -config ./openssl_AC1.cnf
# pem passphrase: pene

openssl x509 -in ac1cert.pem -text -noout

You should get something like this:
```
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            22:82:41:ef:ae:4a:bd:90:2e:f4:2f:e9:0a:13:1b:a9:88:f7:c7:03
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C = ES, ST = MADRID, O = UC3M, OU = INF, CN = AC1, emailAddress = AC1@CRIPTO.INF.UC3M.ES
        Validity
            Not Before: Nov 17 16:42:43 2022 GMT
            Not After : Nov 12 16:42:43 2023 GMT
        Subject: C = ES, ST = MADRID, O = UC3M, OU = INF, CN = AC1, emailAddress = AC1@CRIPTO.INF.UC3M.ES
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:d6:16:6c:a1:c3:cd:2e:fd:07:4e:1a:76:5e:2b:
                    df:dc:de:2c:f8:f8:9f:37:39:d1:cf:58:06:1a:33:
                    99:92:44:8a:a8:60:b4:52:18:3d:8e:e5:41:e6:06:
                    1a:08:42:fb:98:27:62:03:3e:29:24:f6:fc:7d:fa:
                    02:22:bf:d3:16:48:34:1f:51:7d:14:22:9e:7a:bd:
                    d6:6c:4b:be:b2:ee:5a:9c:1a:7f:75:12:cd:a2:64:
                    aa:64:62:f1:23:6d:4e:07:32:d5:f9:01:42:92:1c:
                    18:fa:13:5b:77:fa:f7:63:17:c4:44:12:ab:59:9b:
                    0e:1c:0d:a8:e2:fd:0c:d0:16:a4:8f:40:ce:19:71:
                    65:91:54:0f:6a:6c:ab:ee:4f:81:81:e8:3d:85:ec:
                    97:e0:62:74:81:e1:64:ee:fb:ba:57:7e:8a:34:d7:
                    ab:d2:23:bf:a8:05:66:96:a2:dd:31:89:6d:d3:48:
                    56:a8:00:d4:da:1e:8d:0d:fb:bf:99:9c:d8:f9:82:
                    03:11:88:6f:7e:39:bc:cd:76:84:f6:22:68:1c:b6:
                    c1:a9:ec:d3:61:33:af:42:7a:75:61:d3:5a:7e:cf:
                    b8:c0:1f:93:cb:71:ec:a1:3f:1a:6e:9d:3a:66:d0:
                    cb:ec:be:5a:ea:78:33:79:8d:98:7e:97:98:f0:2c:
                    9e:8f
                Exponent: 65537 (0x10001)
        X509v3 extensions:
            X509v3 Subject Key Identifier: 
                64:7A:95:87:16:55:6B:89:EC:D1:6D:00:BC:6C:3D:3F:5D:EC:37:2D
            X509v3 Authority Key Identifier: 
                keyid:64:7A:95:87:16:55:6B:89:EC:D1:6D:00:BC:6C:3D:3F:5D:EC:37:2D
                DirName:/C=ES/ST=MADRID/O=UC3M/OU=INF/CN=AC1/emailAddress=AC1@CRIPTO.INF.UC3M.ES
                serial:22:82:41:EF:AE:4A:BD:90:2E:F4:2F:E9:0A:13:1B:A9:88:F7:C7:03

            X509v3 Basic Constraints: 
                CA:TRUE
            X509v3 Key Usage: 
                Certificate Sign, CRL Sign
            Netscape Cert Type: 
                SSL CA, S/MIME CA
    Signature Algorithm: sha256WithRSAEncryption
         6a:8e:c6:d2:bc:79:1c:14:ad:82:ef:27:ca:a6:30:22:a2:b3:
         4c:ca:49:a9:5d:a7:cb:9b:03:ef:e4:60:ee:36:ac:15:69:37:
         46:c4:c2:b2:69:a5:a2:d8:e0:2c:18:df:fc:26:5c:ed:25:53:
         1f:4f:43:40:94:ed:59:45:14:3b:c0:76:6b:ba:4b:dc:5a:74:
         b8:7f:34:0d:7c:aa:d3:01:77:28:91:23:24:2a:c4:a9:dc:cf:
         73:d5:4b:e9:49:9a:71:a0:b7:c8:09:04:db:92:e4:10:2f:44:
         7b:7e:0e:08:2a:10:43:d6:6c:d9:27:58:8d:ea:c2:03:d8:a8:
         65:d5:08:b5:8b:be:80:c5:81:8c:5f:14:99:81:a9:60:32:08:
         2d:87:7b:af:45:4c:c9:dd:7a:c1:86:4e:6c:de:23:0c:2c:77:
         68:e9:39:a0:d6:ba:62:96:b2:b6:01:de:b4:5d:15:5f:72:c8:
         17:a8:c3:4e:4c:8a:a2:03:5c:d2:e6:c5:20:d8:04:9f:07:bb:
         3c:28:e3:4c:de:3e:66:80:3d:5b:aa:47:a2:00:50:3b:16:8c:
         52:50:f4:93:92:1b:77:4f:dd:ea:d5:8c:dd:48:e8:dd:17:08:
         ca:f4:62:4f:3a:50:23:b6:74:f3:4d:82:c9:9e:79:f4:15:5f:
         ef:e9:84:fc
```

# 2. Creating the end user's keys and the Certificate Signing Request (CSR)

Before coding anything in Python, prepare a folder for the end user entity (here named as A). 

Go back to the PKI folder and create a folder for A. 

In [ ]:
cd ..
mkdir A

Now write code in the next cell to create an RSA private key for the end user. Serialize and save the private key in a file <code>PKI/A/Akey.pem</code>.  

NOTE DOWN THE PASSWORD FOR THE PRIVATE KEY if you think that you may forget it (if you do not remember it, you'll have to repeat the process AGAIN if you need to use the private key).

In [4]:
KEY_SIZE = 2048  # bits
PUBLIC_EXPONENT = 65537
PASSWORD = b"pene"

priv_key = rsa.generate_private_key(PUBLIC_EXPONENT, KEY_SIZE)

encoding = serialization.Encoding.PEM
format = serialization.PrivateFormat.TraditionalOpenSSL
encryption_algorithm = serialization.BestAvailableEncryption(PASSWORD)

pem_priv_key = priv_key.private_bytes(encoding, format, encryption_algorithm)

with open("PKI/A/Akey.pem", 'wb') as pem_out: 
    pem_out.write(pem_priv_key)

Now write code to create a Certificate Signing Request (CSR) for entity A. Follow the [tutorial provided](https://cryptography.io/en/latest/x509/tutorial/#creating-a-certificate-signing-request-csr) to create the CSR (remember that you should  already have created the private key). 

Make sure you change the COMMON NAME so it contains the NIA of the students working collaboratively in this lab (or just yours if you are working by yourself).

When you follow the tutorial, notice that instead of requesting a certificate for a web server, you’ll request an end user certificate, so you’ll have to:
- use a common name that is not a domain name (not <code>u“mysite.com”</code> )
- do not add the <code>DNSName</code> extension

Save the CSR in a file <code>PKI/A/Acsr.pem</code>.

In [6]:
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"ES"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"Madrid"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"Alcorcón"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"UC3M"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"Acsr"),
])).sign(load_pem_private_key(pem_priv_key, PASSWORD), hashes.SHA256())  # Sign the CSR with our private key

# Write our CSR out to disk.
with open("PKI/A/Acsr.pem", "wb") as f:
    f.write(csr.public_bytes(serialization.Encoding.PEM))

Now, using the terminal, go to A's folder and copy A's CSR in the folder <code>solicitudes</code> of AC1. 

Use command <code>req</code> to see the contents of the CSR.

In [ ]:
cd A
cp ./Acsr.pem ../AC1/solicitudes/

openssl req -in ../AC1/solicitudes/Acsr.pem -text -noout

Here you can see an output similar to the one you should get:
```
Certificate Request:
    Data:
        Version: 1 (0x0)
        Subject: C = ES, ST = MADRID, L = LEGANES, O = UC3M, CN = A_NAME
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:e2:8f:41:35:2c:64:ee:76:8b:5b:c5:a3:d5:7f:
                    d2:a0:01:b1:93:e3:59:7d:0b:4a:0a:2c:b8:56:c5:
                    b0:0e:98:ec:e8:c6:d5:51:bd:e2:a8:be:c0:a9:1c:
                    dc:00:fb:97:0b:ad:20:bb:c7:e7:48:10:c9:70:17:
                    5d:88:f1:87:bb:36:dd:65:78:4f:9f:04:35:f7:82:
                    25:a4:4e:63:8f:7a:c0:69:9d:6d:39:47:94:59:cb:
                    2c:72:f1:53:dd:a6:fb:b0:37:c3:7a:ef:60:c9:96:
                    f6:ea:45:71:bb:9c:73:fe:4f:3e:14:5f:a2:cd:d1:
                    d2:b4:f2:5e:7b:f1:0d:ba:72:93:44:75:67:af:18:
                    e9:bc:1f:f0:a2:92:62:4f:8a:a1:7b:83:2f:9f:56:
                    8d:ae:6a:28:20:8d:a3:83:e7:68:b8:9e:cf:98:e5:
                    4a:43:d0:57:db:73:29:4b:31:7a:af:4b:a2:a8:6b:
                    fa:fe:98:d8:37:03:4c:b2:4a:f4:88:3d:06:f1:ab:
                    b9:43:46:10:41:a1:b0:70:7d:e8:c0:ce:87:24:bf:
                    2a:bf:11:69:d0:e0:db:02:19:d5:b5:b1:ea:70:a0:
                    f9:26:6e:aa:fd:3e:7a:8d:5e:d4:28:e3:cf:48:00:
                    10:c3:a4:41:c4:96:e4:11:bc:0b:b8:cb:b5:db:61:
                    30:d7
                Exponent: 65537 (0x10001)
        Attributes:
            a0:00
    Signature Algorithm: sha256WithRSAEncryption
         74:ec:62:3d:68:b2:b8:80:63:ac:a8:0f:67:ca:9b:58:75:87:
         ce:cd:43:80:4c:fa:b3:0e:72:80:a7:68:99:96:f9:8e:61:cf:
         14:e3:bc:97:50:e2:0d:6c:3c:2c:7e:26:93:07:2a:e8:1a:0f:
         f3:4f:61:79:54:f3:4c:91:11:5f:58:f8:eb:0c:89:ff:d6:32:
         f0:a7:a2:a7:5d:b6:94:ef:37:c1:3f:fe:4e:4c:dd:9c:59:ae:
         7d:18:b5:3e:fc:ed:f7:e5:cd:9c:96:ea:d5:e9:d8:7a:11:2d:
         a7:6b:6c:bb:87:14:9f:54:98:5c:08:0a:96:2e:ae:fb:39:df:
         7c:58:da:c1:58:00:e4:89:53:d3:b0:08:46:ae:ed:39:63:54:
         f6:f6:8d:f4:23:c3:92:62:7f:61:b7:06:c2:c0:36:d0:3f:ca:
         50:5c:36:60:da:bc:f0:59:14:03:84:2a:e2:42:02:66:6e:03:
         c2:8d:a5:22:d7:96:2d:3f:10:2e:17:01:c5:bb:d5:9c:52:35:
         c9:11:f9:bc:fe:7c:71:16:95:20:19:7d:ab:f2:ca:25:0d:3c:
         f2:4f:dc:e1:8b:d4:e6:5c:c9:e5:ee:a9:6b:13:a9:7b:b6:7c:
         71:8f:c7:f8:86:69:d9:63:94:21:c7:26:56:0c:59:e2:cb:8c:
         f5:69:2b:b4
```

# 3. Issuing the end user's certificate

Now go to the AC1 folder and you are going to review A's CSR and then, if everything is OK, you'll issue A's certificate. Notice that requested the pass phrase is the one protecting AC1's private key. 

In [ ]:
cd ../AC1

touch index.txt  # database file

openssl req -in ./solicitudes/Acsr.pem -verify -text -noout -config openssl_AC1.cnf
openssl ca -in ./solicitudes/Acsr.pem -extensions usr_cert -notext -config openssl_AC1.cnf

You should get something like this:
```
Using configuration from openssl_AC1.cnf
Enter pass phrase for .//privado/ca1key.pem:
Check that the request matches the signature
Signature ok
Certificate Details:
        Serial Number: 1 (0x1)
        Validity
            Not Before: Jan 20 09:57:20 2023 GMT
            Not After : Jan 20 09:57:20 2024 GMT
        Subject:
            countryName               = ES
            stateOrProvinceName       = Madrid
            organizationName          = UC3M
            commonName                = Acsr
        X509v3 extensions:
            X509v3 Basic Constraints:
                CA:FALSE
            Netscape Comment:
                OpenSSL Generated Certificate
            X509v3 Subject Key Identifier:
                A4:D4:AA:1B:12:9F:EC:63:DE:C9:7F:76:53:D4:CF:5B:CE:D6:1A:AD
            X509v3 Authority Key Identifier:
                keyid:07:04:5B:7D:D7:A9:A2:B3:47:D1:34:C1:BB:EE:59:5D:00:41:7F:70
                DirName:/C=ES/ST=MADRID/O=UC3M/OU=INF/CN=AC1/emailAddress=AC1@CRIPTO.INF.UC3M.ES
                serial:3F:A5:D8:01:77:60:4E:45:C5:64:64:9C:DF:0B:79:E3:DB:80:9F:72

Certificate is to be certified until Jan 20 09:57:20 2024 GMT (365 days)
Sign the certificate? [y/n]:y


1 out of 1 certificate requests certified, commit? [y/n]y
Write out database with 1 new entries
-----BEGIN CERTIFICATE-----
MIIENTCCAx2gAwIBAgIBATANBgkqhkiG9w0BAQsFADBwMQswCQYDVQQGEwJFUzEP
MA0GA1UECBMGTUFEUklEMQ0wCwYDVQQKEwRVQzNNMQwwCgYDVQQLEwNJTkYxDDAK
BgNVBAMTA0FDMTElMCMGCSqGSIb3DQEJARYWQUMxQENSSVBUTy5JTkYuVUMzTS5F
UzAeFw0yMzAxMjAwOTU3MjBaFw0yNDAxMjAwOTU3MjBaMDwxCzAJBgNVBAYTAkVT
MQ8wDQYDVQQIDAZNYWRyaWQxDTALBgNVBAoMBFVDM00xDTALBgNVBAMMBEFjc3Iw
ggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQCbqweBIwh824HCFUOBd/Jd
uk9DP+OR4pRZMyA312mKTgUdWfmjPWgWV9X7OJHSPHF6iScG7UsZSS40BKzL/OEg
vCQGNqMfySfh67/jRPEpV78bqkGkLIURQnuTvDnuvj9E0QOFBxdJ3jJ5gI0riND4
NGob4FdUBmAYuo+KrB7c9O+BzjVehc43yeSjf3H55d/aC56g+lk4qKcg4mi52HUf
JiMPIxGq51UGCQ+bJstkCXIzW419zHiWjCaLgDYMiF3ZVLjJ6UNb6il52vYo31IA
CffFLZLZM1G/a4Lq3+gM4lcYNK9eNKxtQy4lU7DSFIucKmz5u8k9vdqL90xE+uz/
AgMBAAGjggEMMIIBCDAJBgNVHRMEAjAAMCwGCWCGSAGG+EIBDQQfFh1PcGVuU1NM
IEdlbmVyYXRlZCBDZXJ0aWZpY2F0ZTAdBgNVHQ4EFgQUpNSqGxKf7GPeyX92U9TP
W87WGq0wga0GA1UdIwSBpTCBooAUBwRbfdeporNH0TTBu+5ZXQBBf3ChdKRyMHAx
CzAJBgNVBAYTAkVTMQ8wDQYDVQQIEwZNQURSSUQxDTALBgNVBAoTBFVDM00xDDAK
BgNVBAsTA0lORjEMMAoGA1UEAxMDQUMxMSUwIwYJKoZIhvcNAQkBFhZBQzFAQ1JJ
UFRPLklORi5VQzNNLkVTghQ/pdgBd2BORcVkZJzfC3nj24CfcjANBgkqhkiG9w0B
AQsFAAOCAQEADuT5M163xSrKDcZoOXlrNIP7ItXwRWBO4x13efDqKml5VCW/OE+g
an1F3IVdKzk/eY/ZcnFpm6oix6Q8AqS8eFTwnS9DqOyNLkdiv7qrBr+ke557dKch
tfUtacmh8sIRePpvqEiqL2k47/qFIM9gIJuGHYeYJW6HJybq6MlB7tXNBaahw9Xy
EC0U36c5HWBGV2cZEoO40n+ud/vNX2/Mc8DTgL7jl80D6qSa5JbLnVX1r9NQflgr
5YenEtkgf7xwTAceSb+BW0gsL1OaLtaj84F8n1NUQ7pJEdaUPw5y+3GEDrvOGUGQ
887aUOHtQug4kVHXb0fPEgs77DuC6XdNqw==
-----END CERTIFICATE-----
Data Base Updated
```

Finally, print the contents of the issued certificate and copy it --the file <code>nuevoscerts/01.pem</code> (or named with the corresponding serial number that you have used for A's certificate)-- to A's folder as <code>Acert.pem</code>.

In [ ]:
openssl x509 -in ./nuevoscerts/01.pem -text -noout

cp ./nuevoscerts/01.pem ../A/Acert.pem

You should get something like this:
```
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number: 1 (0x1)
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C = ES, ST = MADRID, O = UC3M, OU = INF, CN = AC1, emailAddress = AC1@CRIPTO.INF.UC3M.ES
        Validity
            Not Before: Nov 17 17:28:14 2022 GMT
            Not After : Nov 17 17:28:14 2023 GMT
        Subject: C = ES, ST = MADRID, O = UC3M, CN = A_NAME
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:e2:8f:41:35:2c:64:ee:76:8b:5b:c5:a3:d5:7f:
                    d2:a0:01:b1:93:e3:59:7d:0b:4a:0a:2c:b8:56:c5:
                    b0:0e:98:ec:e8:c6:d5:51:bd:e2:a8:be:c0:a9:1c:
                    dc:00:fb:97:0b:ad:20:bb:c7:e7:48:10:c9:70:17:
                    5d:88:f1:87:bb:36:dd:65:78:4f:9f:04:35:f7:82:
                    25:a4:4e:63:8f:7a:c0:69:9d:6d:39:47:94:59:cb:
                    2c:72:f1:53:dd:a6:fb:b0:37:c3:7a:ef:60:c9:96:
                    f6:ea:45:71:bb:9c:73:fe:4f:3e:14:5f:a2:cd:d1:
                    d2:b4:f2:5e:7b:f1:0d:ba:72:93:44:75:67:af:18:
                    e9:bc:1f:f0:a2:92:62:4f:8a:a1:7b:83:2f:9f:56:
                    8d:ae:6a:28:20:8d:a3:83:e7:68:b8:9e:cf:98:e5:
                    4a:43:d0:57:db:73:29:4b:31:7a:af:4b:a2:a8:6b:
                    fa:fe:98:d8:37:03:4c:b2:4a:f4:88:3d:06:f1:ab:
                    b9:43:46:10:41:a1:b0:70:7d:e8:c0:ce:87:24:bf:
                    2a:bf:11:69:d0:e0:db:02:19:d5:b5:b1:ea:70:a0:
                    f9:26:6e:aa:fd:3e:7a:8d:5e:d4:28:e3:cf:48:00:
                    10:c3:a4:41:c4:96:e4:11:bc:0b:b8:cb:b5:db:61:
                    30:d7
                Exponent: 65537 (0x10001)
        X509v3 extensions:
            X509v3 Basic Constraints: 
                CA:FALSE
            Netscape Comment: 
                OpenSSL Generated Certificate
            X509v3 Subject Key Identifier: 
                D4:55:E0:24:DB:2B:2F:F2:4F:FE:A3:C9:52:99:40:DF:95:87:D8:68
            X509v3 Authority Key Identifier: 
                keyid:64:7A:95:87:16:55:6B:89:EC:D1:6D:00:BC:6C:3D:3F:5D:EC:37:2D
                DirName:/C=ES/ST=MADRID/O=UC3M/OU=INF/CN=AC1/emailAddress=AC1@CRIPTO.INF.UC3M.ES
                serial:22:82:41:EF:AE:4A:BD:90:2E:F4:2F:E9:0A:13:1B:A9:88:F7:C7:03

    Signature Algorithm: sha256WithRSAEncryption
         b2:b8:70:84:f0:ea:bd:21:cb:32:65:ab:1e:49:64:5b:33:7b:
         1f:d5:61:ba:f0:79:ab:20:59:be:c6:38:0f:79:c4:33:18:84:
         ec:79:0a:70:54:cd:bb:7b:82:2f:ad:45:48:19:3e:3f:79:d3:
         89:44:af:b6:fb:78:4e:1e:85:4c:13:06:02:2c:00:b3:3a:13:
         83:dc:0a:1a:de:e0:78:97:91:29:b8:17:22:23:c9:4e:e2:5a:
         5c:a4:21:53:49:7f:94:e5:b0:fb:82:c4:d5:83:ef:ff:37:96:
         d3:8c:43:19:c8:8b:31:86:76:d7:f9:4a:c7:0f:e6:41:2f:d0:
         02:f2:fb:3a:29:ac:a3:2b:b7:e2:08:da:3e:51:ff:be:12:ab:
         4a:b8:c8:48:89:4f:62:3f:ee:af:c4:0e:4b:e4:38:5d:1e:57:
         34:6b:fb:f7:31:74:28:ff:a0:7a:e2:f7:19:de:e9:5a:64:15:
         81:4b:73:47:97:26:9a:14:9c:de:1e:93:43:9a:31:31:54:bd:
         ea:c8:10:5e:a7:ff:72:d0:59:16:e0:f7:da:4b:24:f6:64:4b:
         73:8e:84:77:f9:cf:a6:52:11:fa:81:ba:88:78:66:52:f9:dc:
         cf:d3:d9:f9:76:c2:75:c4:91:e6:f1:f4:6d:7d:99:bb:96:d6:
         a6:57:96:f2
```

# 4. Using the end user's certificate

Now, write code to load and unserialize A's certificate. Notice that your are provided with several auxiliary functions to do this (<code>load_pem</code> and <code>unserialize_pemcert</code>). Once you have the certificate object, print the value of some of its attributes (consult [reference provided](https://cryptography.io/en/latest/x509/reference/#x-509-certificate-object)) such as the serial number, the fingerprint, the subject, the validity period...   

In [20]:
def unserialize_pemcert(pemlines):
    certificate = load_pem_x509_certificate(pemlines)
    return certificate

def save_pem(pem, filename): 
    with open(filename, 'wb') as pem_out: 
        pem_out.write(pem) 

def load_pem(filename): 
    with open(filename, 'rb') as pem_in: 
        pemlines = pem_in.read() 
        return pemlines

pemlines = load_pem("./PKI/A/Acert.pem")

Acert = unserialize_pemcert(pemlines)

print("version:", Acert.version)
print("serial_number:", Acert.serial_number)
print("public_key.n:", hex(Acert.public_key().public_numbers().n))
print("not_valid_before:", Acert.not_valid_before)
print("not_valid_after:", Acert.not_valid_after)
print("issuer:", Acert.issuer)
print("signature_hash_algorithm:", Acert.signature_hash_algorithm.name)
print("signature:", Acert.signature.hex())
print("tbs_certificate_bytes:", Acert.tbs_certificate_bytes.hex())

version: Version.v3
serial_number: 1
public_key.n: 0x9bab078123087cdb81c215438177f25dba4f433fe391e29459332037d7698a4e051d59f9a33d681657d5fb3891d23c717a892706ed4b19492e3404accbfce120bc240636a31fc927e1ebbfe344f12957bf1baa41a42c8511427b93bc39eebe3f44d10385071749de3279808d2b88d0f8346a1be05754066018ba8f8aac1edcf4ef81ce355e85ce37c9e4a37f71f9e5dfda0b9ea0fa5938a8a720e268b9d8751f26230f2311aae75506090f9b26cb640972335b8d7dcc78968c268b80360c885dd954b8c9e9435bea2979daf628df520009f7c52d92d93351bf6b82eadfe80ce2571834af5e34ac6d432e2553b0d2148b9c2a6cf9bbc93dbdda8bf74c44faecff
not_valid_before: 2023-01-20 09:57:20
not_valid_after: 2024-01-20 09:57:20
issuer: <Name(C=ES,ST=MADRID,O=UC3M,OU=INF,CN=AC1,1.2.840.113549.1.9.1=AC1@CRIPTO.INF.UC3M.ES)>
signature_hash_algorithm: sha256
signature: 0ee4f9335eb7c52aca0dc66839796b3483fb22d5f045604ee31d7779f0ea2a69795425bf384fa06a7d45dc855d2b393f798fd97271699baa22c7a43c02a4bc7854f09d2f43a8ec8d2e4762bfbaab06bfa47b9e7b74a721b5f52d69c9a1f2c21178fa6fa848aa2f6938effa8520c

From now on, the public key will be inside the certificate and you can access to it with <code>Acert.public_key()</code>. 

# 5. Encrypting a message for your teacher

Load the cert in <code>files/teachercert.pem</code> and use the public key within it to encrypt a message to your teacher. Use RSA with OAEP-SHA256 padding (parameters <code>mgf = asym_padding.MGF1(algorithm=utils.hashes.SHA256())</code>, <code>algorithm = utils.hashes.SHA256()</code>, and <code>label = None</code>). 

Once you have the encrypted message, encode it in base64 and save it in a file using `wb` mode (name the file `files/secret_message_from_YOUR-NAME-SURNAME.enc`).

In [22]:
MESSAGE = b"en el aeropuerto hay avione"

# load public key
t_cert = unserialize_pemcert(load_pem("files/teachercert.pem"))
t_pub_key = t_cert.public_key()

# encrypt
padder = asym_padding.OAEP(asym_padding.MGF1(algorithm=utils.hashes.SHA256()), utils.hashes.SHA256(), label=None)
ct = t_pub_key.encrypt(MESSAGE, padder)

# save file
with open("files/secret_message_from_LUIS-DANIEL-CASAIS-MEZQUIDA", "wb") as f:
    f.write(base64.b64encode(ct))

# 6. Signing a PDF file within Acrobat Reader

Next, you'll use the <code>OpenSSL</code> tool to export the end user's certificate together with its certification chain (in this case it only contains the certificate of AC1) AND the end user's private key.

First, move in the terminal to A's folder, then copy AC1's certificate to A's folder and finally, use command <code>pkcs12</code> to export A's cert, AC1's cert and A's private key into a single file in PKCS12 format. You'll be asked for an 'Export Password'; this password will be used to protect the private key inside the PKCS12 file, so make sure you will be able to recall it afterwards if you do not want to repeat the process. 

In [ ]:
cd ../A
cp ../AC1/ac1cert.pem .

# password: pene
openssl pkcs12 -export -in Acert.pem -inkey Akey.pem -certfile ac1cert.pem -out Acert_with_sk.p12

Now, download the PKCS12 file to a computer with Adobe Reader installed. Download also the provided PDF file in Aula Global and open it within the Adobe Reader program. 

Search for 'Tools' or 'Herramientas' and look for the tool named 'Certificates' or 'Certificados'. Open it and select then 'Digitally Sign' or 'Firmar digitalmente'. Follow the instructions and provide the required information when requested:
- The area where the signature's graphical mark will be placed 
- The file containing the PKCS12 certificate
- The password protecting the private key (previously named 'Export Password') 

Save the signed PDF file and check the signature you've just generated. Check also that you can see in the certificates associated with the signature the data present in AC1 and A's certificates.

# 7. FNMT-RCM Certification Authority

The organization Fábrica Nacional de Moneda y Timbre (FNMT) from the Spanish Government issues
[private individual certificates](https://www.sede.fnmt.gob.es/en/certificados/persona-fisica). 
You may request one if you want and satisfy the requirements, although in this lab you are not requested so. 
In this lab you are requested to have a look at some documentation that a real PKI has to publish (Certification Practices Statement) and also check the AC RAIZ FNMT-RCM certificate (the root certificate) against the one that is embedded in the Firefox browser (it should be installed in the lab’s Windows computer).

Access the Certification Practices Statement file in [this URL](https://www.sede.fnmt.gob.es/documents/10445900/10536309/dgpc_english.pdf).
Have a look at the index of the file to grasp the type of information that a document of this type should contain. 
Then, check the information provided from page 14 to 17 related to the FNMT-RCM root certificate. In particular, look for the fingerprint of the certificate. 

On the other hand, open a Firefox browser or a Chrome browser. Open the Preferences panel in any of the browsers. Look for the Privacy and Security configuration and specifically for the the place where you can check the certificates. Search for the embbeded FNMT-RCM root certificate and view its contents. Look for the fingerprint and check that it is the same that the one specified in the Certification Practices Statement. 

# 8. Web server's certificates 

Open the Mozilla Firefox browser or the Chrome web browser and access the web pages listed next. Right-click on the lock icon that
you’ll find next to the URL to its left and click on the “Certificate” link to get more information on this
web server certificate:
- https://www.google.com
- https://www.uc3m.es
- https://stackoverflow.com/
- https://administracion.gob.es/
- https://www.wikipedia.org/

Find out for each certificate:
- the certified common name (and associated DNS names)
- the validity period
- the type and length of the certified keys
- the key usages
- which entity has issued each certificate (and how long the certificate chain is) 
- the type and length of the keys of the issuing authority, and 
- the validity period of the certificate of the issuing authority.

Open a browser and open the following web pages:
- https://letsencrypt.org 
- https://letsencrypt.org/getting-started/
- https://letsencrypt.org/how-it-works/

Now answer the following questions:  
**Q:** Which is the purpose of this organization?  
**A:** To provide encryption certificates for websites.  
**Q:** Which are the pros of the server public key certificates issued by Let’s encrypt?  
**A:** Adds extra security.  